# Дообучение модели машинного перевода opus на аннотациях из сферы когнитивных наук

Исходные данные – сборники тезисов конференции "Когнитивная наука в Москве". Конференция проходит раз в 2 года, и с 2015 года в числе требований к публикации тезисов – авторы должны написать аннотацию и на русском, и на английском языке. Я дообучила модель opus на 5 сборниках тезисов этой конференции (с 2015 по 2023 год, всего 505 докладов).

# Достаем текст из пдф

Изначально сборники тезисов каждый год выпускаются в формате одного большого пдф-файла. Для начала достала из каждой пдф текст (делала это в notebook локально, потом запускать код заново уже не было необходимости, поэтому путь отличается)

In [ ]:
#!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 9.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.9 MB/s eta 0:00:00a 0:00:01


In [ ]:
import pdfplumber
import os

input_folder = "/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings"
output_folder = "/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt"

for filename in os.listdir(input_folder):
    pdf_path = os.path.join(input_folder, filename)
    txt_path = os.path.join(output_folder, filename.replace(".pdf", ".txt"))

    print(f"processing {filename}")

    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:  #пропуск пустых страниц на всякий случай
                full_text += text + "\n\n"

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"{txt_path} saved")

processing 2015.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2015.txt saved
processing 2017.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2017.txt saved
processing 2013.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2013.txt saved
processing 2011.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2011.txt saved
processing 2021.pdf


Cannot set gray non-stroke color because /'P0' is an invalid float value


/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2021.txt saved
processing 2023.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2023.txt saved
processing 2019.pdf
/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2019.txt saved


# Собираем датасет

Так как все тезисы с одного года были собраны в один большой .txt, нужно было поделить файл на отдельные тексты:


In [ ]:
import re
def split_into_papers(text):
    candidates = re.split(r'\w+@\w+.\w+', text) #у каждой работы есть email :) оказался самым простым и надёжным способом разделить тексты
    return [c.strip() for c in candidates if len(c.strip()) > 200]  #на случай, если попадет какой нибудь мусор

In [ ]:
with open('/Users/kotyarodionova/Desktop/учеба/compling/mlproject/proceedings_txt/2023.txt', encoding='utf-8') as f:  #подставляла каждый текст по очереди, потому что важно было проследить, что ничего не ломается (а если ломается - то на каком документе)
                text = f.read()

In [ ]:
papers = split_into_papers(text)

In [ ]:
del papers[0] #это самая первая часть, до текстов – содержание, слово авторов и редакторов, титульник и прочее

Структура самих текстов была следующая: аффилиация и аннотация на языке доклада (рус или англ), сами тезисы, аффилиация и аннотация на противоположном языке. Например, статья на русском языке выглядит вот так:

1. Название, авторы, аффилиация email автора + аннотация на русском
2. Текст тезисов
3. Name, authors, affiliation, author's email, abstract (in english)


Т.к. в одной статье email указан дважды, функция разделила каждую из статей на часть на основном языке (первая аннотация + тезисы) и на втором языке (вторая аннотация). Склеиваем:

In [ ]:
joined_papers = [papers[i] + papers[i+1] for i in range(0, len(papers), 2)]

In [ ]:
abstracts_ru = []
abstracts_en = []

Немного очистки текста:

In [ ]:
def clean_abstract(abstract):
    try:
        abstract = re.sub(r'-\n', '', abstract) #убираем переносы от верстки
    except: pass
    try:
        abstract = re.sub(r'\n(\d{1,3})\n', '', abstract) #убираем номера страниц
    except: pass
    try:
        abstract = re.sub(r'\n([A-ZА-ЯЁ][A-Za-zА-Яа-яЁё.\s,]*?(?:et al\.)?)\n', '', abstract) #убираем упоминание автора внизу каждой страницы
    except: pass
    return abstract

Ищем аннотации. Они расположены от слова "Аннотация" до словосочетания "Ключевые слова"

In [ ]:
for text in joined_papers:
    match = re.search(r"(аннотация\.?\s*)(.*?)(?=\n?ключевые слова:?)", text, re.IGNORECASE | re.DOTALL)
    if match:
        abstract = match.group(2).strip()
        abstracts_ru.append(clean_abstract(abstract))
    else: abstracts_ru.append('ru abstract error!')
    match = re.search(r"(abstract\.?\s*)(.*?)(?=\n?keywords:?)", text, re.IGNORECASE | re.DOTALL)
    if match:
        abstract = match.group(2).strip()
        abstracts_en.append(clean_abstract(abstract))
    else: abstracts_en.append('en abstract error!')

Проверим, что ничего не потерялось

In [ ]:
print(len(abstracts_en))
print(len(abstracts_ru))

505
505


In [ ]:
import pandas as pd
df = pd.DataFrame(list(zip(abstracts_en, abstracts_ru)), columns = ['abstract_en', 'abstract_ru'])

In [ ]:
df.to_csv('abstracts.csv')

# Подгружаем датасет

Посмотрела датасет глазами: в сборнике  2015 г. появилось около 7 abstract error из-за ошибок вёрстки, добавила их в датасет вручную, чтобы не терять такие ценные данные. В том же году около 5 раз в аннотацию проникали имена авторов и названия университетов, тоже почистила вручную

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/abstracts_corrected.csv')

In [ ]:
df #уже очищенный красивый датасет

,abstract_en,abstract_ru
0,This study is an analysis of visual complexity...,Настоящая работа представляет собой анализ одн...
1,The current study investigated the prevention ...,Данная работа направлена на поиск способов пре...
2,In this study 59 participants reported their e...,В исследовании сопоставлялись самоотчеты 59-ти...
3,"Ensemble perception, including the ability to ...","Восприятие зрительных ансамблей, в том числе с..."
4,This study discusses characteristics of oculom...,В ходе исследования были получены характеристи...
...,...,...
500,We developed an instrument for assessing readi...,Мы разработали инструментарий для оценки навык...
501,A new method has been developed for assessing ...,В рамках настоящего проекта было положено нача...
502,"A pilot study was conducted, during which a mo...","Проведено пилотное исследование, в ходе которо..."
503,Traditional studies on cortical activity relie...,В традиционных исследованиях кортикальной акти...


# Модель и обучение (рус-англ)

In [ ]:
!pip install transformers[torch] datasets sentencepiece sacrebleu

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
train_val = dataset.train_test_split(test_size=0.2)
val_test = train_val["test"].train_test_split(test_size=0.5)

train_dataset = train_val["train"]
val_dataset = val_test["train"]
test_dataset = val_test["test"]

In [ ]:
#сохраняю датасеты, чтобы примеры не мешались каждый раз, если придется перезапускать код!
from datasets import DatasetDict
full_dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

full_dataset.save_to_disk("/content/drive/MyDrive/dataset_split")

Saving the dataset (0/1 shards):   0%|          | 0/404 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
!pip install sacremoses #модель opus просит при загрузке

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.9 MB/s eta 0:00:00


In [ ]:
max_input_length = 512
max_target_length = 512

def preprocess(examples):
    inputs = examples["abstract_ru"]
    targets = examples["abstract_en"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = full_dataset.map(preprocess, batched=True) #применяем ко всем трём сетам

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/translation",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1, #боюсь больше просто не потянет колаб
    per_device_eval_batch_size=1,
    num_train_epochs=6, #число не из головы. мне пришлось заново переобучать модель, и именно на 6 эпохе лосс остановился в прошлый раз
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipython-input-12-3531509262.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,2.518800,2.744537
2,2.731400,2.654047
3,2.480300,2.597121
4,2.559100,2.579970
5,1.980200,2.579455
6,1.854000,2.572809


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2424, training_loss=2.2924124323102113, metrics={'train_runtime': 10722.5681, 'train_samples_per_second': 0.226, 'train_steps_per_second': 0.226, 'total_flos': 149731726196736.0, 'train_loss': 2.2924124323102113, 'epoch': 6.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/translation")
tokenizer.save_pretrained("/content/drive/MyDrive/translation")

('/content/drive/MyDrive/translation/tokenizer_config.json',
 '/content/drive/MyDrive/translation/special_tokens_map.json',
 '/content/drive/MyDrive/translation/vocab.json',
 '/content/drive/MyDrive/translation/source.spm',
 '/content/drive/MyDrive/translation/target.spm',
 '/content/drive/MyDrive/translation/added_tokens.json')

# Модель и обучение (англ-рус)

Конечно, можно было свернуть всё обучение в функции и просто подставить туда обратную модель. Но дообучить и обратную модель я догадалась не сразу, а cell outputs терять не хочется, поэтому код тут просто дублируется :)

In [2]:
!pip install sacremoses #модель opus просит при загрузке
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name_reverse = "Helsinki-NLP/opus-mt-en-ru"
tokenizer_reverse = AutoTokenizer.from_pretrained(model_name_reverse)
model_reverse = AutoModelForSeq2SeqLM.from_pretrained(model_name_reverse)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
max_input_length = 512
max_target_length = 512

def preprocess_reverse(examples):
    inputs = examples["abstract_en"]
    targets = examples["abstract_ru"]
    model_inputs = tokenizer_reverse(inputs, max_length=max_input_length, truncation=True)

    with tokenizer_reverse.as_target_tokenizer():
        labels = tokenizer_reverse(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets_reverse = dataset.map(preprocess_reverse, batched=True) #применяем ко всем трём сетам

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [4]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args_reverse = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/translation_enru",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1, #боюсь больше просто не потянет колаб
    per_device_eval_batch_size=1,
    num_train_epochs=6, #число не из головы. мне пришлось заново переобучать модель, и именно на 6 эпохе лосс остановился в прошлый раз
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

trainer_reverse = Seq2SeqTrainer(
    model=model_reverse,
    args=training_args_reverse,
    train_dataset=tokenized_datasets_reverse["train"],
    eval_dataset=tokenized_datasets_reverse["validation"],
    tokenizer=tokenizer_reverse,
)
trainer_reverse.train()

/tmp/ipython-input-4-290083407.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_reverse = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.433700,2.622237
2,2.672000,2.531482
3,2.301100,2.491832
4,2.425400,2.481922
5,1.766600,2.478379
6,1.597700,2.478780


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2424, training_loss=2.181708892973343, metrics={'train_runtime': 8486.4783, 'train_samples_per_second': 0.286, 'train_steps_per_second': 0.286, 'total_flos': 132669196075008.0, 'train_loss': 2.181708892973343, 'epoch': 6.0})

# Для продолжения тренировки после перезапуска среды (в итоговой версии проекта не использовала):

In [ ]:
!cp -r "/content/drive/MyDrive/dataset_split" "/content/dataset_split" #не всегда даёт загрузить датасет, protocol error

In [1]:
from google.colab import drive
drive.mount('/content/drive')
from datasets import load_from_disk

dataset = load_from_disk("/content/drive/MyDrive/dataset_split")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

path_to_checkpoint = "/content/drive/MyDrive/translation/checkpoint-1212"
model = AutoModelForSeq2SeqLM.from_pretrained(path_to_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(path_to_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 512

def preprocess(examples):
    inputs = examples["abstract_ru"]
    targets = examples["abstract_en"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
  output_dir="/content/drive/MyDrive/translation",
  eval_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  num_train_epochs=3,
  weight_decay=0.01,
  save_total_limit=2,
  save_strategy="epoch",
  predict_with_generate=True,
  logging_dir='./logs',
  logging_steps=10,
  report_to="none"
)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["validation"],
  tokenizer=tokenizer
)
trainer.train() #не получилось from_checkpoint :(

/tmp/ipython-input-15-4084615001.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.546100,2.380595
2,1.611500,2.377134


Epoch,Training Loss,Validation Loss
1,1.546100,2.380595
2,1.611500,2.377134
3,1.618800,2.369080


TrainOutput(global_step=1212, training_loss=2.0192290855319586, metrics={'train_runtime': 4464.4954, 'train_samples_per_second': 0.271, 'train_steps_per_second': 0.271, 'total_flos': 75407707275264.0, 'train_loss': 2.0192290855319586, 'epoch': 3.0})

# Evaluation (рус-англ)

Этот код с загрузкой модели и токенизацией повторяется, потому что я писала разные части в разные дни, и мне было важно, чтобы трейн, продолжение трейна, и evaluation могли воспроизводиться независимо друг от друга

In [ ]:
!pip install -U "datasets[fsspec]" fsspec #для загрузки датасета

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
from datasets import load_from_disk
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

dataset = load_from_disk("/content/drive/MyDrive/dataset_split")

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

path_to_checkpoint = "/content/drive/MyDrive/translation/checkpoint-2424"
model = AutoModelForSeq2SeqLM.from_pretrained(path_to_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(path_to_checkpoint)

max_input_length = 512
max_target_length = 512

def preprocess(examples):
    inputs = examples["abstract_ru"]
    targets = examples["abstract_en"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess, batched=True)

training_args = Seq2SeqTrainingArguments(
  output_dir="/content/drive/MyDrive/translation",
  eval_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  num_train_epochs=3,
  weight_decay=0.01,
  save_total_limit=2,
  save_strategy="epoch",
  predict_with_generate=True,
  logging_dir='./logs',
  logging_steps=10,
  report_to="none"
)

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_datasets["train"],
  eval_dataset=tokenized_datasets["validation"],
  tokenizer=tokenizer
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

/tmp/ipython-input-3-47408416.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


Считаем стандартный BLEU, и, так как не все считают его достоверной метрикой (https://aclanthology.org/2020.acl-main.448.pdf), посмотрим ещё chrF

In [ ]:
from sacrebleu import corpus_bleu
from sacrebleu import corpus_chrf

#делаем предсказания и декодим их в строки
predictions = trainer.predict(tokenized_datasets["test"])
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

#декодим референсы (человеческие аннотации на английском)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

#чистим
decoded_preds = [pred.strip() for pred in decoded_preds]
decoded_labels = [label.strip() for label in decoded_labels]

#наконец, считаем bleu
bleu = corpus_bleu(decoded_preds, [decoded_labels])
print(f"\nBLEU для модели, которая дообучалась 6 эпох: {bleu.score:.2f}")

chrf = corpus_chrf(decoded_preds, [decoded_labels])
print(f"\nchrF для модели, которая дообучалась 6 эпох: {chrf.score:.2f}")


BLEU для модели, которая дообучалась 6 эпох: 24.47

chrF для модели, которая дообучалась 6 эпох: 56.98


In [ ]:
model_base = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
tokenizer_base = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

def preprocess_base(examples):
    inputs = examples["abstract_ru"]
    targets = examples["abstract_en"]
    model_inputs = tokenizer_base(inputs, max_length=max_input_length, truncation=True)

    with tokenizer_base.as_target_tokenizer():
        labels = tokenizer_base(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets_base = dataset.map(preprocess_base, batched=True)

trainer_base = Seq2SeqTrainer(
  model=model_base,
  args=training_args,
  train_dataset=tokenized_datasets_base["train"],
  eval_dataset=tokenized_datasets_base["validation"],
  tokenizer=tokenizer_base
)

predictions_base = trainer_base.predict(tokenized_datasets_base["test"])
decoded_preds_base = tokenizer_base.batch_decode(predictions_base.predictions, skip_special_tokens=True)

#декодим референсы (человеческие аннотации на английском)
decoded_labels_base = tokenizer_base.batch_decode(predictions_base.label_ids, skip_special_tokens=True)

#чистим
decoded_preds_base = [pred.strip() for pred in decoded_preds_base]
decoded_labels_base = [label.strip() for label in decoded_labels_base]

#наконец, считаем bleu и chrf
bleu_base = corpus_bleu(decoded_preds_base, [decoded_labels_base])
print(f"\nBLEU для базовой модели: {bleu_base.score:.2f}")

chrf_base = corpus_chrf(decoded_preds_base, [decoded_labels_base])
print(f"\nchrF для базовой модели: {chrf_base.score:.2f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

...

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

/tmp/ipython-input-6-1846864690.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_base = Seq2SeqTrainer(



BLEU для базовой модели: 18.30

chrF для базовой модели: 50.25


Небольшое предупреждение: Значения bleu для базовой модели в коде и в таблице/презентации отличаются (в таблице – 16.87). Дело в том, что изначально я не считала chrF, поэтому после добавления новой метрики эту ячейку приходилось редактировать и перезапускать. Между двумя запусками кода, видимо, базовую модель доработали (или что-то еще случилось), и bleu поднялся с 16.87 при первом запуске кода до 17.65 сейчас. Я подумала, что раз для дообучения я брала базовую модель ДО этого изменения, справедливо будет докладывать первый результат

In [ ]:
model_3epochs = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/checkpoint-1212")
tokenizer_3epochs = AutoTokenizer.from_pretrained("/content/drive/MyDrive/checkpoint-1212")

def preprocess_3epochs(examples):
    inputs = examples["abstract_ru"]
    targets = examples["abstract_en"]
    model_inputs = tokenizer_3epochs(inputs, max_length=max_input_length, truncation=True)

    with tokenizer_3epochs.as_target_tokenizer():
        labels = tokenizer_3epochs(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets_3epochs = dataset.map(preprocess_3epochs, batched=True)

trainer_3epochs = Seq2SeqTrainer(
  model=model_3epochs,
  args=training_args,
  train_dataset=tokenized_datasets_3epochs["train"],
  eval_dataset=tokenized_datasets_3epochs["validation"],
  tokenizer=tokenizer_3epochs
)

predictions_3epochs = trainer_3epochs.predict(tokenized_datasets_3epochs["test"])
decoded_preds_3epochs = tokenizer_3epochs.batch_decode(predictions_3epochs.predictions, skip_special_tokens=True)

#декодим референсы (человеческие аннотации на английском)
decoded_labels_3epochs = tokenizer_3epochs.batch_decode(predictions_3epochs.label_ids, skip_special_tokens=True)

#чистим
decoded_preds_3epochs = [pred.strip() for pred in decoded_preds_3epochs]
decoded_labels_3epochs = [label.strip() for label in decoded_labels_3epochs]

#наконец, считаем bleu и chrf
bleu_3epochs = corpus_bleu(decoded_preds_3epochs, [decoded_labels_3epochs])
print(f"\nBLEU для модели, которая дообучалась 3 эпохи: {bleu_3epochs.score:.2f}")

chrf_3epochs = corpus_chrf(decoded_preds_3epochs, [decoded_labels_3epochs])
print(f"\nchrF для модели, которая дообучалась 3 эпохи: {chrf_3epochs.score:.2f}")

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

/tmp/ipython-input-7-3454511123.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_3epochs = Seq2SeqTrainer(



BLEU для модели, которая дообучалась 3 эпохи: 24.30

chrF для модели, которая дообучалась 3 эпохи: 56.64


# Мой собственый пример :)

Наши с коллегами тезисы на ту же самую конференцию 2025 года (ещё не опубликованы):

In [ ]:
test_text = "Распространённость деменции порождает потребность совершенствовать раннюю диагностику ухудшения когнитивных функций на этапах субъективного когнитивного снижения (СКС) и мягкого когнитивного снижения (МКС) для своевременного вмешательства специалистов. Ведётся поиск надёжных и доступных инструментов выявления когнитивных нарушений на ранних стадиях ухудшения когнитивных функций, в том числе при помощи речевых показателей. Широко используется задание на направленные вербальные ассоциации (НВА, также применяется вариант «вербальная беглость»), но в клинической практике, как правило, учитывается только общее количество правильно названных слов. В данном исследовании анализируются результаты выполнения пациентами с СКС и МКС заданий на фонематические (ФВА; n=113) и семантические вербальные ассоциации (СВА; n=93) посредством автоматизированного выделения семантических (сходных по значению) и фонетических (сходных по звучанию) кластеров в ответе. В каждом типе задания сравнивались две линейных модели со смешанными эффектами для предсказания клинической группы и общего балла по когнитивной шкале MoCA: модель, включающая в себя в качестве предиктора только общее количество названных слов, и модель, включающая как общее количество слов, так и характеристики выделенных кластеров. Модель для СВА, включающая в себя характеристики кластеров, значимо лучше предсказывала клиническую группу (χ²(4)=12.65, p=.013), но не общий балл по MoCA. В задании на ФВА характеристики кластеров не улучшали предсказательную способность моделей. Таким образом, характеристики кластеров могут повышать диагностическую точность задания на СВА."

Дообученная модель:

In [ ]:
import torch
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, padding=True).to(model.device)

model.eval()
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=512)

english_translation = tokenizer.decode(output[0], skip_special_tokens=True)
print(english_translation)

The spread of dementia leads to the need to improve early diagnostics of cognitive function deterioration in the stages of subjective cognitive decrease (CCS) and mild cognitive decrease (CCS) for the timely intervention of specialists. A search is made for reliable and available tools for the detection of cognitive impairments in the early stages of cognitive deterioration, including through speech indicators. A task aimed at sent verbal associations (VATs) is also widely used, but only the total number of correctly named words is usually taken into account in clinical practice. In this study, we analyzed the results of tasks performed by patients with CCS and ICS on phonological (FBA; n = 113) and semantic verbal associations (SBA; n = 93) via the automated selection of semantic (like) and phonical (like) clusters in the response. In each type of task, we compared two linear models with mixed effects for predicting the clinical group and the overall score on the cognitive scale of Mo

Базовая модель:

In [ ]:
inputs = tokenizer_base(test_text, return_tensors="pt", truncation=True, padding=True).to(model_base.device)

model_base.eval()
with torch.no_grad():
    output = model_base.generate(**inputs, max_new_tokens=512)

english_translation_base = tokenizer_base.decode(output[0], skip_special_tokens=True)
print(english_translation_base)

The spread of dementia creates a need to improve the early diagnosis of cognitive degradation during the subjective cognitive reduction (CSS) and soft cognitive reduction (CLS) phases for timely intervention by specialists. A search is under way for reliable and accessible tools to detect cognitive impairments in the early stages of cognitive degradation, including through voice indicators. The assignment to sent notes verbale associations (NVA) is also widely used, but clinical practice tends to take into account only the total number of words correctly quoted. This study compares the results of patients &apos; performance with the CSS and ISS on the genomic (FBA; n=113) and semantic notes verbale associations (SBA; n=93) by the automated identification of semantic (same in terms of accuracy) models and the radiant (same in terms of magnitude) clusters in the response.


3 эпохи:

In [ ]:
inputs = tokenizer_3epochs(test_text, return_tensors="pt", truncation=True, padding=True).to(model_3epochs.device)

model_3epochs.eval()
with torch.no_grad():
    output = model_3epochs.generate(**inputs, max_new_tokens=512)

english_translation_3epochs = tokenizer_3epochs.decode(output[0], skip_special_tokens=True)
print(english_translation_3epochs)

The spread of dementia creates a need to improve early diagnostics of cognitive function deterioration during the stages of subjective cognitive decrease (CSR) and mild cognitive decrease (CRM) for the timely intervention of experts. We search for reliable and accessible tools for the detection of cognitive impairments at the early stages of cognitive deterioration, including through speech indicators. A task aimed at verbal associations (NVA) is also widely used, but only the total number of correctly named words is usually taken into account in clinical practice. In this study, we analyzed the results of patients with CCS and ICS performing tasks on the phonological (FBA; n = 113) and semantic verbal associations (SBA; n = 93) via the automated selection of semantic (like) and phonical (like) clusters in the response. In each type of task, we compared two linear models with mixed effects for predicting the clinical group and the overall score on the cognitive scale (SBA: the total nu

In [ ]:
import pandas as pd
df_results = pd.DataFrame([
    {"model": "base", "translation": english_translation_base, "BLEU": bleu_base.score, "chrF": chrf_base},
    {"model": "3 epochs", "translation": english_translation_3epochs, "BLEU": bleu_3epochs.score, "chrF": chrf_3epochs},
    {"model": "6 epochs", "translation": english_translation, "BLEU": bleu.score, "chrF": chrf}
])

In [ ]:
df_results.to_csv('mt_finetuning_results.csv') #ссылка есть в презентации

# Evaluation + пример + табличка (англ-рус)

здесь тоже код повторяется, потому что модель англ-рус была обучена позже

In [6]:
from sacrebleu import corpus_bleu
from sacrebleu import corpus_chrf

#6 эпох:
predictions_reverse = trainer_reverse.predict(tokenized_datasets_reverse["test"])
decoded_preds_reverse = tokenizer_reverse.batch_decode(predictions_reverse.predictions, skip_special_tokens=True)
decoded_labels_reverse = tokenizer_reverse.batch_decode(predictions_reverse.label_ids, skip_special_tokens=True)
decoded_preds_reverse = [pred.strip() for pred in decoded_preds_reverse]
decoded_labels_reverse = [label.strip() for label in decoded_labels_reverse]
bleu_reverse = corpus_bleu(decoded_preds_reverse, [decoded_labels_reverse])
print(f"\nBLEU для модели (англ-рус), которая дообучалась 6 эпох: {bleu_reverse.score:.2f}")
chrf_reverse = corpus_chrf(decoded_preds_reverse, [decoded_labels_reverse])
print(f"\nchrF для модели (англ-рус), которая дообучалась 6 эпох: {chrf_reverse.score:.2f}")

#base:
model_base_reverse = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
tokenizer_base_reverse = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
def preprocess_base_reverse(examples):
    inputs = examples["abstract_en"]
    targets = examples["abstract_ru"]
    model_inputs = tokenizer_base_reverse(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_base_reverse.as_target_tokenizer():
        labels = tokenizer_base_reverse(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_datasets_base_reverse = dataset.map(preprocess_base_reverse, batched=True)
trainer_base_reverse = Seq2SeqTrainer(
  model=model_base_reverse,
  args=training_args_reverse,
  train_dataset=tokenized_datasets_base_reverse["train"],
  eval_dataset=tokenized_datasets_base_reverse["validation"],
  tokenizer=tokenizer_base_reverse
)
predictions_base_reverse = trainer_base_reverse.predict(tokenized_datasets_base_reverse["test"])
decoded_preds_base_reverse = tokenizer_base_reverse.batch_decode(predictions_base_reverse.predictions, skip_special_tokens=True)
decoded_labels_base_reverse = tokenizer_base_reverse.batch_decode(predictions_base_reverse.label_ids, skip_special_tokens=True)
decoded_preds_base_reverse = [pred.strip() for pred in decoded_preds_base_reverse]
decoded_labels_base_reverse = [label.strip() for label in decoded_labels_base_reverse]
bleu_base_reverse = corpus_bleu(decoded_preds_base_reverse, [decoded_labels_base_reverse])
print(f"\nBLEU для базовой модели (англ-рус): {bleu_base_reverse.score:.2f}")
chrf_base_reverse = corpus_chrf(decoded_preds_base_reverse, [decoded_labels_base_reverse])
print(f"\nchrF для базовой модели (англ-рус): {chrf_base_reverse.score:.2f}")

#3 эпохи:
model_3epochs_reverse = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/checkpoint-1212-reverse")
tokenizer_3epochs_reverse = AutoTokenizer.from_pretrained("/content/drive/MyDrive/checkpoint-1212-reverse")
def preprocess_3epochs_reverse(examples):
    inputs = examples["abstract_en"]
    targets = examples["abstract_ru"]
    model_inputs = tokenizer_3epochs_reverse(inputs, max_length=max_input_length, truncation=True)
    with tokenizer_3epochs_reverse.as_target_tokenizer():
        labels = tokenizer_3epochs_reverse(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_datasets_3epochs_reverse = dataset.map(preprocess_3epochs_reverse, batched=True)
trainer_3epochs_reverse = Seq2SeqTrainer(
  model=model_3epochs_reverse,
  args=training_args_reverse,
  train_dataset=tokenized_datasets_3epochs_reverse["train"],
  eval_dataset=tokenized_datasets_3epochs_reverse["validation"],
  tokenizer=tokenizer_3epochs_reverse
)
predictions_3epochs_reverse = trainer_3epochs_reverse.predict(tokenized_datasets_3epochs_reverse["test"])
decoded_preds_3epochs_reverse = tokenizer_3epochs_reverse.batch_decode(predictions_3epochs_reverse.predictions, skip_special_tokens=True)
decoded_labels_3epochs_reverse = tokenizer_3epochs_reverse.batch_decode(predictions_3epochs_reverse.label_ids, skip_special_tokens=True)
decoded_preds_3epochs_reverse = [pred.strip() for pred in decoded_preds_3epochs_reverse]
decoded_labels_3epochs_reverse = [label.strip() for label in decoded_labels_3epochs_reverse]
bleu_3epochs_reverse = corpus_bleu(decoded_preds_3epochs_reverse, [decoded_labels_3epochs_reverse])
print(f"\nBLEU для модели, которая дообучалась 3 эпохи (англ-рус): {bleu_3epochs_reverse.score:.2f}")
chrf_3epochs_reverse = corpus_chrf(decoded_preds_3epochs_reverse, [decoded_labels_3epochs_reverse])
print(f"\nchrF для модели, которая дообучалась 3 эпохи (англ-рус): {chrf_3epochs_reverse.score:.2f}")


BLEU для модели (англ-рус), которая дообучалась 6 эпох: 17.83

chrF для модели (англ-рус), которая дообучалась 6 эпох: 53.62


Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

/tmp/ipython-input-6-1350387763.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer_3epochs_reverse = Seq2SeqTrainer(



BLEU для модели, которая дообучалась 3 эпохи (англ-рус): 17.21

chrF для модели, которая дообучалась 3 эпохи (англ-рус): 53.14


In [7]:
test_text_reverse = 'Due to the growing number of dementia diagnoses, it is crucial to detect cognitive decline at earlier stages, such as subjective cognitive decline (SCD) and mild cognitive impairment (MCI). Nowadays, there is a growing interest in finding reliable and accessible instruments of cognitive decline early detection, including language-based measures. The verbal fluency (VF) task is among most used tools; however, only total word count is usually employed to evaluate task performance. In this study, performance of patients with SCD and MCI on phonemic (PVF) and semantic (SVF) verbal fluency tasks was examined via automatic analysis of clustering. For each task type, two linear mixed effects models were built to predict total MoCA score and diagnosis: the first model included only total word count as a predictor, whereas the second model included both total word count and clustering characteristics. The analysis showed that the SVF model with clustering characteristics improved predictive power for the clinical group (χ²(4)=12.65, p=0.013) but not for total MoCA score. As for PVF, however, no significant improvement was found both for diagnosis and total MoCA score. As a result, examining clustering characteristics in SVF task performance along with the total word count might improve diagnostics.'

In [10]:
import torch

#6 эпох
inputs = tokenizer_reverse(test_text_reverse, return_tensors="pt", truncation=True, padding=True).to(model_reverse.device)
model_reverse.eval()
with torch.no_grad():
    output = model_reverse.generate(**inputs, max_new_tokens=512)
english_translation_reverse = tokenizer_reverse.decode(output[0], skip_special_tokens=True)
print(english_translation_reverse)

#base
inputs = tokenizer_base_reverse(test_text_reverse, return_tensors="pt", truncation=True, padding=True).to(model_base_reverse.device)
model_base_reverse.eval()
with torch.no_grad():
    output = model_base_reverse.generate(**inputs, max_new_tokens=512)
english_translation_base_reverse = tokenizer_base_reverse.decode(output[0], skip_special_tokens=True)
print(english_translation_base_reverse)

#3 эпохи
inputs = tokenizer_3epochs_reverse(test_text_reverse, return_tensors="pt", truncation=True, padding=True).to(model_3epochs_reverse.device)
model_3epochs_reverse.eval()
with torch.no_grad():
    output = model_3epochs_reverse.generate(**inputs, max_new_tokens=512)
english_translation_3epochs_reverse = tokenizer_3epochs_reverse.decode(output[0], skip_special_tokens=True)
print(english_translation_3epochs_reverse)

В связи с увеличением числа диагностаций слабоумия крайне важно обнаружить когнитивное упадок на ранних этапах, таких как субъективное когнитивное упадок (МКН) и слабое когнитивное нарушение (ММК). В настоящее время растет интерес к поиску надежных и доступных инструментов раннего обнаружения когнитивного упадка, включая языковые меры. Для предсказания общего количества баллов и диагностирования МПР были построены две модели линейного смешанного эффекта: первая модель включала только общий подсчет слов в качестве предиктора, в то время как вторая модель включала как общий подсчет слов, так и кластерные характеристики. Анализ показал, что модель SVF с категорическими характеристиками улучшала предикальную мощность для клинической группы (2 (4) = 12.65, p = 0.013), но не для общей оценки МПР. Однако значимого улучшения как для диагностирования, так и для всего МПР не обнаружено. Анализ показал, что модель SVF с категорическими характеристиками улучшала мощность предсказания для клиническ

In [11]:
import pandas as pd
df_results_reverse = pd.DataFrame([
    {"model": "base", "translation": english_translation_base_reverse, "BLEU": bleu_base_reverse.score, "chrF": chrf_base_reverse},
    {"model": "3 epochs", "translation": english_translation_3epochs_reverse, "BLEU": bleu_3epochs_reverse.score, "chrF": chrf_3epochs_reverse},
    {"model": "6 epochs", "translation": english_translation_reverse, "BLEU": bleu_reverse.score, "chrF": chrf_reverse}
])

df_results_reverse.to_csv('mt_finetuning_results_reverse.csv') #ссылка есть в презентации